# MScFE 640 - Portfolio Theory and Asset Pricing
# Group Assignment - Group 22 - Submission 1

## Python Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import dcor
import itertools

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## Part-1: Data Importing
### 1.1 & 1.2 Data Import for US Sector ETFs

Monthly price and volume data for the period 2014-2019 is imported for the following ETFs covering 11 US sectors. 
- IYR: iShares U.S. Real Estate ETF
- IYZ: iShares U.S. Telecommunications ETF
- XLB: Materials Select Sector SPDR Fund
- XLE: Energy Select Sector SPDR Fund
- XLF: Financial Select Sector SPDR Fund
- XLI: Industrial Select Sector SPDR Fund
- XLK: Technology Select Sector SPDR Fund
- XLP: Consumer Staples Select Sector SPDR Fund
- XLU: Utilities Select Sector SPDR Fund
- XLV: Health Care Select Sector SPDR Fund
- XLY: Consumer Discretionary Select Sector SPDR Fund

As required, 9 of the ETFs are SPDRs and the remaining 2 (IYR - US Real Estate  and IYZ - US Telecommunications) are iShares ETFs since some data for the historical period under analysis was not available for the S&P counterparts of these ETFs.

In [2]:
etfs_lst = ['IYR', 'IYZ', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
etfs_dict = {}

for etf in etfs_lst:
    etfs_dict[str(etf).lower()] = pd.read_csv('Datasets/Monthly ETF Data/' + etf + '.csv', index_col = 0, parse_dates = True)

Regarding the choice of frequency, the analysis was first carried out using daily and weekly ETF datasets and appropriate transformations of the economic indicators was undertaken for the same. 

However, the resulting linear regression models had very low explanatory power (based on R-squared). R-squared values improved significantly when monthly data was used, and this has been the basis for choosing to use monthly data in this final version.

Nevertheless, draft versions of the analysis with daily and weekly data have also been provided for your reference.

### 1.3 Data Import for Leading, Coincidental and Lagging Economic Indicators (LEI, CEI, LAG)

Monthly data for the period 2014-2019 for 19 economic indicators provided by The Conference Board has been imported below from https://fred.stlouisfed.org/. These are categorised into Leading, Coincidental and Lagging indicators.

**Leading Economic Indicators (LEI)**
Leading indicators have pivotal moments that lead those of the overall economy. The following 9 indicators have been used:

<table> 
    <tr>
        <td>
            <b>Indicator</b>
        </td>
        <td>
            <b>Abbreviation</b>
        </td>
        <td>
            <b>Units</b>
        </td>
        <td>
            <b>Frequency</b>
        </td>
    </tr>
        <td>
        Average Weekly Hours of All Employees, Manufacturing
        </td>
        <td>
        AWHAEMAN
        </td>
        <td>
        Hours, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    <tr>
        <td>
        4-Week Moving Average of Initial Unemployment Claims
        </td>
        <td>
        IC4WSA
        </td>
        <td>
        Number, Seasonally Adjusted
        </td>
        <td>
        Monthly, Average
        </td>
    </tr>
    <tr>
        <td>
        Manufacturers' New Orders: Durable Goods
        </td>
        <td>
        DGORDER
        </td>
        <td>
        Millions of Dollars, Seasonally Adjusted
        </td>
        <td>
        Monthly, Average
        </td>
    </tr>
    <tr>
        <td>
        Manufacturers' New Orders: Nondefense Capital Goods Excluding Aircraft
        </td>
        <td>
        NEWORDER
        </td>
        <td>
        Millions of Dollars, Seasonally Adjusted
        </td>
        <td>
        Monthly, Average
        </td>
    </tr>
    <tr>
        <td>
        New Privately-Owned Housing Units Authorized in Permit-Issuing Places: Total Units
        </td>
        <td>
        PERMIT
        </td>
        <td>
        Thousands of Units, Seasonally Adjusted Annual Rate
        </td>
        <td>
        Monthly
        </td>
    </tr>
    <tr>
        <td>
        S&P 500
        </td>
        <td>
        SP500
        </td>
        <td>
        Index, Not Seasonally Adjusted
        </td>
        <td>
        Monthly, Average
        </td>
    </tr>
    <tr>
        <td>
        Chicago Fed National Financial Conditions Index Nonfinancial Leveral Subindex
        </td>
        <td>
        NFCINONFINLEVERAGE
        </td>
        <td>
        Index, Not Seasonally Adjusted
        </td>
        <td>
        Monthly, Average
        </td>
    </tr>
    <tr>
        <td>
        Market Yield on U.S. Treasury Securities at 10-year Constant Maturity Minus Federal Funds Effective Rate
        </td>
        <td>
        DGS10_FEDFUNDS
        </td>
        <td>
        %-%, Not Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
    <tr>
        <td>
        University of Michigan: Consumer Sentiment
        </td>
        <td>
        UMCSENT
        </td>
        <td>
        Index 1966:Q1=100, Not Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
</table> 

**Coincidental Economic Indicators (CEI)**
Coincidental indicators have pivotal moments that are usually near to those of the overall economy. The following 4 indicators have been used:

<table> 
    <tr>
        <td>
            <b>Indicator</b>
        </td>
        <td>
            <b>Abbreviation</b>
        </td>
        <td>
            <b>Units</b>
        </td>
        <td>
            <b>Frequency</b>
        </td>
    </tr>
        <td>
        All Employees, Total Nonfarm
        </td>
        <td>
        PAYEMS
        </td>
        <td>
        Thousands of Persons, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    <tr>
        <td>
        Real personal income excluding current transfer receipts
        </td>
        <td>
        W875RX1
        </td>
        <td>
        Billions of Chained 2012 Dollars, Seasonally Adjusted Annual Rate
        </td>
        <td>
        Monthly
        </td>
    </tr>
    <tr>
        <td>
        Industrial Production: Total Index
        </td>
        <td>
        INDPRO
        </td>
        <td>
        Index 2017=100, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
    <tr>
        <td>
        Real Manufacturing and Trade Industries Sales
        </td>
        <td>
        CMRMTSPL
        </td>
        <td>
        Millions of Chained 2012 Dollars, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
</table>

**Lagging Economic Indicators (CEI)**
have pivotal moments reflecting the overall economy’s historical performance — lagging indicators are used to categorize the economy’s past condition. The following 6 indicators have been used:

<table> 
    <tr>
        <td>
            <b>Indicator</b>
        </td>
        <td>
            <b>Abbreviation</b>
        </td>
        <td>
            <b>Units</b>
        </td>
        <td>
            <b>Frequency</b>
        </td>
    </tr>
        <td>
        All Employees, Total NonfarmManufacturers: Inventories to Sales Ratio
        </td>
        <td>
        MNFCTRIRSA
        </td>
        <td>
        Ratio, Seasonally Adjusted
        </td>
        <td>
        Monthly, End of Period
        </td>
    <tr>
        <td>
        Average Weeks Unemployed
        </td>
        <td>
        UEMPMEAN
        </td>
        <td>
        Weeks, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
    <tr>
        <td>
        Consumer Installment Credit Outstanding to Personal Income Ratio
        </td>
        <td>
        CREDIT_BY_INC
        </td>
        <td>
        Ratio, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
    <tr>
        <td>
        Commercial and Industrial Loans, All Commercial Banks
        </td>
        <td>
        BUSLOANS
        </td>
        <td>
        Billions of U.S. Dollars, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
    <tr>
        <td>
        Bank Prime Loan Rate
        </td>
        <td>
        DPRIME
        </td>
        <td>
        Percent, Not Seasonally Adjusted
        </td>
        <td>
        Monthly, Average
        </td>
    </tr>
    <tr>
        <td>
        Consumer Price Index for All Urban Consumers: Services Less Energy Services in U.S. City Average
        </td>
        <td>
        CUSR0000SASLE
        </td>
        <td>
        Index 1982-1984=100, Seasonally Adjusted
        </td>
        <td>
        Monthly
        </td>
    </tr>
</table>

In [3]:
lei_lst = ['AWHAEMAN', 'IC4WSA', 'DGORDER', 'NEWORDER', 'PERMIT', 'SP500', 'NFCINONFINLEVERAGE', 'DGS10_FEDFUNDS', 'UMCSENT'] #ISM® new orders index missing
lei_dict = {}

cei_lst = ['PAYEMS', 'W875RX1', 'INDPRO', 'CMRMTSPL']
cei_dict = {}

lag_lst = ['MNFCTRIRSA', 'UEMPMEAN', 'TOTALSL', 'W875RX1', 'BUSLOANS', 'DPRIME', 'CUSR0000SASLE']
lag_dict = {}

for lei in lei_lst:
    lei_dict[str(lei).lower()] = pd.read_csv('Datasets/Economic Indicators (Use With Monthly Data)/' + lei + '.csv', index_col = 0, parse_dates = True)
    
for cei in cei_lst:
    cei_dict[str(cei).lower()] = pd.read_csv('Datasets/Economic Indicators (Use With Monthly Data)/' + cei + '.csv', index_col = 0, parse_dates = True)

for lag in lag_lst:
    lag_dict[str(lag).lower()] = pd.read_csv('Datasets/Economic Indicators (Use With Monthly Data)/' + lag + '.csv', index_col = 0, parse_dates = True)
    
lag_dict['credit_by_inc'] = pd.DataFrame(lag_dict['totalsl']['TOTALSL']/ lag_dict['w875rx1']['W875RX1'], columns = ['CREDIT_BY_INC'])
    

As shown in the last line of the above code, an estimate of Consumer Installment Credit Outstanding to Personal Income Ratio has been obtained by dividing Total Consumer Credit Owned and Securitized (TOTALSL) by Real Personal Income Excluding Current Transfer Receipts (W875RX1).

## Part-2: Data Processing
### 2.1 Monthly Returns Computation

Adj Close price for each ETF has been used to compute monthly log returns for each ETF, as shown below.

In [4]:
logrtn_dict = {}

for etf in etfs_dict:
    df = etfs_dict[etf]
    logrtn_dict[str(etf) + ' log return'] = pd.DataFrame(np.log(df['Adj Close']/df['Adj Close'].shift(1)).iloc[1:])

As an example, the monthly log returns for the XLF fund have been displayed in the below dataframe.

In [5]:
logrtn_dict['xlf log return']

,Adj Close
Date,
2014-01-01,-0.032592
2014-02-01,0.029937
2014-03-01,0.029066
2014-04-01,-0.013457
2014-05-01,0.014916
...,...
2019-08-01,-0.048224
2019-09-01,0.039335
2019-10-01,0.029778


## Part-3: Category 1 Models - Linear Regression
### 3.1 Linear Regression of the 11 ETFs' Log Returns on Leading Economic Indicators (LEI)

In [6]:
lst_for_Xlei = []
for k, v in lei_dict.items():
    lst_for_Xlei.append(lei_dict[k])
        
Xlei = reduce(lambda left, right: pd.merge(left, right, on=['DATE'], how='outer'), lst_for_Xlei)
Xlei

,AWHAEMAN,IC4WSA,DGORDER,NEWORDER,PERMIT,SP500,NFCINONFINLEVERAGE,DGS10_FEDFUNDS,UMCSENT
DATE,,,,,,,,,
2014-01-01,40.7,329000.0,226704,68237,976.0,1822.356667,-1.173738,2.788095,81.2
2014-02-01,40.7,333250.0,230482,67386,1039.0,1817.034737,-1.169715,2.639474,81.6
2014-03-01,41.0,325350.0,235368,68683,1067.0,1863.523333,-1.176168,2.643333,80.0
2014-04-01,40.9,319000.0,235724,66486,1090.0,1864.263333,-1.186338,2.615238,84.1
2014-05-01,41.1,320150.0,229731,65819,1018.0,1889.767143,-1.191008,2.469048,81.9
...,...,...,...,...,...,...,...,...,...
2019-08-01,40.5,215150.0,245614,65593,1479.0,2897.498182,-0.668378,-0.503636,89.8
2019-09-01,40.5,213125.0,245317,65187,1439.0,2982.156000,-0.697202,-0.340500,93.2
2019-10-01,40.3,214437.5,250673,65642,1509.0,2977.675217,-0.723010,-0.123182,95.5


A cursory glance at the above dataframe reveals that the economic indicators have very different ranges. So, the above dataset is normalised before a multiple linear regression model is fit for each ETF.

In [7]:
Xlei_normalized = StandardScaler().fit_transform(Xlei)
Xlei_normalized = pd.DataFrame(Xlei_normalized, columns = Xlei.columns, index = Xlei.index)

In [8]:
regr_linear = LinearRegression()
Xlei_param_matrix = np.empty((0,9))
Xlei_r_sq = np.empty((0,1))

for k, v in logrtn_dict.items():
    Y = v
    regr_linear.fit(Xlei_normalized, Y)
    
    Xlei_param_matrix = np.append(Xlei_param_matrix, regr_linear.coef_.reshape(1,9), axis = 0)
    Xlei_r_sq = np.append(Xlei_r_sq, np.array(regr_linear.score(Xlei_normalized, Y)).reshape(1,1), axis = 0)

The results of the regression are displayed in the parameter matrix below, which shows the coefficients of the various predictors as well as the R-squared for each of the 11 models.

In [9]:
col = Xlei.columns
ind = [i.split()[0].upper() for i in logrtn_dict]
Xlei_param_matrix = pd.DataFrame(Xlei_param_matrix, index=ind, columns=col)

Xlei_param_matrix.insert(0, 'R-squared', Xlei_r_sq)
Xlei_param_matrix

,R-squared,AWHAEMAN,IC4WSA,DGORDER,NEWORDER,PERMIT,SP500,NFCINONFINLEVERAGE,DGS10_FEDFUNDS,UMCSENT
IYR,0.114582,-0.001593,-0.024406,-0.002864,-0.003617,-0.024734,0.006101,-0.010351,-0.006013,-0.008075
IYZ,0.046510,0.002254,-0.004327,0.002724,-0.010062,-0.014251,0.009426,-0.004626,-0.007264,-0.005115
XLB,0.103479,0.005566,0.016643,-0.007504,-0.013796,-0.011074,0.032939,-0.002712,-0.013925,-0.013626
XLE,0.104510,0.008299,0.076285,-0.000845,-0.011013,-0.000121,0.045219,0.035005,-0.018514,-0.014773
XLF,0.170083,0.000314,0.004838,-0.002830,-0.025116,-0.018455,0.062751,-0.007139,0.012704,-0.024759
XLI,0.139086,0.002384,-0.004769,-0.010855,-0.012932,-0.016416,0.035418,-0.003975,-0.001570,-0.021892
XLK,0.143782,0.003663,0.001972,-0.003227,-0.021573,-0.021499,0.053968,-0.011507,0.003004,-0.016276
XLP,0.119721,-0.003013,-0.015485,-0.007753,-0.002389,-0.020365,0.018404,-0.008721,-0.000522,-0.008387
XLU,0.148679,0.001766,-0.030993,-0.014655,0.008060,-0.012867,-0.009923,-0.010932,-0.010441,-0.007674
XLV,0.149585,0.000497,0.009406,-0.008722,-0.011597,-0.015228,0.053378,-0.007994,0.011154,-0.011821


### 3.2 Linear Regression of the 11 ETFs' Log Returns on Lagging Economic Indicators (LAG)

In [10]:
totalsl = lag_dict.pop('totalsl')
w875rx1 = lag_dict.pop('w875rx1')

In [11]:
lst_for_Xlag = []
for k, v in lag_dict.items():
    lst_for_Xlag.append(lag_dict[k])
        
Xlag = reduce(lambda left, right: pd.merge(left, right, on=['DATE'], how='outer'), lst_for_Xlag)

#with pd.option_context("display.max_rows", 2000, "display.max_columns", 100):
#    display(Xlag)
Xlag

,MNFCTRIRSA,UEMPMEAN,BUSLOANS,DPRIME,CUSR0000SASLE,CREDIT_BY_INC
DATE,,,,,,
2014-01-01,1.30,35.1,1586.7465,3.250000,290.063,0.263490
2014-02-01,1.29,36.5,1619.9242,3.250000,290.590,0.263307
2014-03-01,1.30,35.3,1634.1492,3.250000,291.366,0.263480
2014-04-01,1.30,34.9,1650.9200,3.250000,292.062,0.264791
2014-05-01,1.31,34.4,1663.5430,3.250000,292.806,0.265673
...,...,...,...,...,...,...
2019-08-01,1.48,21.9,2363.1229,5.250000,338.528,0.296922
2019-09-01,1.49,22.3,2368.4184,5.150000,339.445,0.297472
2019-10-01,1.50,22.4,2365.1365,4.988636,340.342,0.298130


Once again, the above dataset is normalised before a multiple linear regression model is fit for each ETF.

In [12]:
Xlag_normalized = StandardScaler().fit_transform(Xlag)
Xlag_normalized = pd.DataFrame(Xlag_normalized, columns = Xlag.columns, index = Xlag.index)

#Xlag_normalized

In [13]:
regr_linear = LinearRegression()
Xlag_param_matrix = np.empty((0,6))
Xlag_r_sq = np.empty((0,1))

for k, v in logrtn_dict.items():
    Y = v
    regr_linear.fit(Xlag_normalized, Y)
    
    Xlag_param_matrix = np.append(Xlag_param_matrix, regr_linear.coef_.reshape(1,6), axis = 0)
    Xlag_r_sq = np.append(Xlag_r_sq, np.array(regr_linear.score(Xlag_normalized, Y)).reshape(1,1), axis = 0)

In [14]:
col = Xlag.columns
ind = [i.split()[0].upper() for i in logrtn_dict]
Xlag_param_matrix = pd.DataFrame(Xlag_param_matrix, index=ind, columns=col)

Xlag_param_matrix.insert(0, 'R-squared', Xlag_r_sq)
Xlag_param_matrix

,R-squared,MNFCTRIRSA,UEMPMEAN,BUSLOANS,DPRIME,CUSR0000SASLE,CREDIT_BY_INC
IYR,0.040511,0.008077,-0.004328,-0.006473,0.028138,-0.040749,0.010535
IYZ,0.020704,0.000201,0.003472,0.032172,0.009508,-0.040144,0.002689
XLB,0.028873,0.000370,0.022917,0.020946,-0.007337,-0.012720,0.022603
XLE,0.031102,-0.010839,0.030115,0.045464,-0.018796,-0.022376,0.032933
XLF,0.077047,-0.000800,0.017857,0.020598,-0.013406,-0.050456,0.063124
XLI,0.015905,0.001412,0.007247,0.011066,-0.006447,-0.013956,0.016950
XLK,0.031851,0.005812,-0.009230,-0.031522,-0.006674,0.016345,0.012296
XLP,0.017474,0.005338,0.002315,0.001055,0.012012,-0.008287,-0.004993
XLU,0.045125,0.004439,0.012269,0.004177,0.015412,0.003240,-0.013345
XLV,0.031734,0.000015,0.005018,-0.031380,-0.010491,0.042703,0.002194


### 3.3 Linear Regression of the 11 ETFs' Log Returns on Coincidental Economic Indicators (CEI)

In [15]:
lst_for_Xcei = []
for k, v in cei_dict.items():
    lst_for_Xcei.append(cei_dict[k])
        
Xcei = reduce(lambda left, right: pd.merge(left, right, on=['DATE'], how='outer'), lst_for_Xcei)
Xcei

,PAYEMS,W875RX1,INDPRO,CMRMTSPL
DATE,,,,
2014-01-01,137548,11800.5,100.0380,1328414.0
2014-02-01,137714,11870.9,100.7918,1339979.0
2014-03-01,137968,11933.0,101.7782,1350652.0
2014-04-01,138293,11965.2,101.8337,1352609.0
2014-05-01,138511,11993.1,102.2325,1356343.0
...,...,...,...,...
2019-08-01,151081,13912.9,102.6882,1493174.0
2019-09-01,151244,13933.7,102.3143,1491257.0
2019-10-01,151337,13947.5,101.4645,1482087.0


The above dataset is normalised before a multiple linear regression model is fit for each ETF.

In [16]:
Xcei_normalized = StandardScaler().fit_transform(Xcei)
Xcei_normalized = pd.DataFrame(Xcei_normalized, columns = Xcei.columns, index = Xcei.index)

#Xcei_normalized

In [17]:
regr_linear = LinearRegression()
Xcei_param_matrix = np.empty((0,4))
Xcei_r_sq = np.empty((0,1))

for k, v in logrtn_dict.items():
    Y = v
    regr_linear.fit(Xcei_normalized, Y)
    
    Xcei_param_matrix = np.append(Xcei_param_matrix, regr_linear.coef_.reshape(1,4), axis = 0)
    Xcei_r_sq = np.append(Xcei_r_sq, np.array(regr_linear.score(Xcei_normalized, Y)).reshape(1,1), axis = 0)

In [18]:
col = Xcei.columns
ind = [i.split()[0].upper() for i in logrtn_dict]
Xcei_param_matrix = pd.DataFrame(Xcei_param_matrix, index=ind, columns=col)

Xcei_param_matrix.insert(0, 'R-squared', Xcei_r_sq)
Xcei_param_matrix

,R-squared,PAYEMS,W875RX1,INDPRO,CMRMTSPL
IYR,0.021508,0.048576,-0.046100,0.006470,-0.005965
IYZ,0.004206,0.016346,-0.023145,0.000663,0.006112
XLB,0.034730,-0.002662,-0.020624,-0.007171,0.026584
XLE,0.083671,-0.002610,-0.071235,-0.008272,0.077749
XLF,0.041653,0.036410,-0.052047,-0.003985,0.018132
XLI,0.037666,-0.018924,-0.007350,-0.007961,0.029974
XLK,0.019776,-0.000393,-0.007471,-0.004512,0.012546
XLP,0.003617,-0.008292,0.000940,0.000828,0.007298
XLU,0.029058,0.061916,-0.043326,0.006389,-0.021313
XLV,0.010836,0.036878,-0.030111,0.004840,-0.009029


### 3.4 Linear Regression of the 11 ETFs' Log Returns on All Economic Indicators (ALL)

In [19]:
lst_for_Xall = [Xlei, Xlag, Xcei]

#for i in [lst_for_Xlei, lst_for_Xlag, lst_for_Xcei]:
#    for j in i:
#            lst_for_Xall.append(j)

Xall = reduce(lambda left, right: pd.merge(left, right, on=['DATE'], how='outer'), lst_for_Xall)
Xall

,AWHAEMAN,IC4WSA,DGORDER,NEWORDER,PERMIT,SP500,NFCINONFINLEVERAGE,DGS10_FEDFUNDS,UMCSENT,MNFCTRIRSA,UEMPMEAN,BUSLOANS,DPRIME,CUSR0000SASLE,CREDIT_BY_INC,PAYEMS,W875RX1,INDPRO,CMRMTSPL
DATE,,,,,,,,,,,,,,,,,,,
2014-01-01,40.7,329000.0,226704,68237,976.0,1822.356667,-1.173738,2.788095,81.2,1.30,35.1,1586.7465,3.250000,290.063,0.263490,137548,11800.5,100.0380,1328414.0
2014-02-01,40.7,333250.0,230482,67386,1039.0,1817.034737,-1.169715,2.639474,81.6,1.29,36.5,1619.9242,3.250000,290.590,0.263307,137714,11870.9,100.7918,1339979.0
2014-03-01,41.0,325350.0,235368,68683,1067.0,1863.523333,-1.176168,2.643333,80.0,1.30,35.3,1634.1492,3.250000,291.366,0.263480,137968,11933.0,101.7782,1350652.0
2014-04-01,40.9,319000.0,235724,66486,1090.0,1864.263333,-1.186338,2.615238,84.1,1.30,34.9,1650.9200,3.250000,292.062,0.264791,138293,11965.2,101.8337,1352609.0
2014-05-01,41.1,320150.0,229731,65819,1018.0,1889.767143,-1.191008,2.469048,81.9,1.31,34.4,1663.5430,3.250000,292.806,0.265673,138511,11993.1,102.2325,1356343.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,40.5,215150.0,245614,65593,1479.0,2897.498182,-0.668378,-0.503636,89.8,1.48,21.9,2363.1229,5.250000,338.528,0.296922,151081,13912.9,102.6882,1493174.0
2019-09-01,40.5,213125.0,245317,65187,1439.0,2982.156000,-0.697202,-0.340500,93.2,1.49,22.3,2368.4184,5.150000,339.445,0.297472,151244,13933.7,102.3143,1491257.0
2019-10-01,40.3,214437.5,250673,65642,1509.0,2977.675217,-0.723010,-0.123182,95.5,1.50,22.4,2365.1365,4.988636,340.342,0.298130,151337,13947.5,101.4645,1482087.0


In [20]:
Xall_normalized = StandardScaler().fit_transform(Xall)
Xall_normalized = pd.DataFrame(Xall_normalized, columns = Xall.columns, index = Xall.index)

#Xall_normalized

In [21]:
regr_linear = LinearRegression()
Xall_param_matrix = np.empty((0,19))
Xall_r_sq = np.empty((0,1))

for k, v in logrtn_dict.items():
    Y = v
    regr_linear.fit(Xall_normalized, Y)
    
    Xall_param_matrix = np.append(Xall_param_matrix, regr_linear.coef_.reshape(1,19), axis = 0)
    Xall_r_sq = np.append(Xall_r_sq, np.array(regr_linear.score(Xall_normalized, Y)).reshape(1,1), axis = 0)

In [22]:
col = Xall.columns
ind = [i.split()[0].upper() for i in logrtn_dict]
Xall_param_matrix = pd.DataFrame(Xall_param_matrix, index=ind, columns=col)

Xall_param_matrix.insert(0, 'R-squared', Xall_r_sq)
Xall_param_matrix

,R-squared,AWHAEMAN,IC4WSA,DGORDER,NEWORDER,PERMIT,SP500,NFCINONFINLEVERAGE,DGS10_FEDFUNDS,UMCSENT,MNFCTRIRSA,UEMPMEAN,BUSLOANS,DPRIME,CUSR0000SASLE,CREDIT_BY_INC,PAYEMS,W875RX1,INDPRO,CMRMTSPL
IYR,0.188660,-0.003832,-0.018236,-0.004585,-0.016479,-0.021388,-0.011037,-0.009227,-0.002603,-0.001736,0.012492,-0.024759,-0.041172,0.037984,0.089434,-0.002219,-0.067305,-0.067414,0.009346,0.035567
IYZ,0.137117,0.012018,-0.009615,0.002842,-0.013527,-0.008750,0.059999,0.008914,-0.000597,-0.002347,0.012298,-0.035882,0.090664,0.026137,0.142344,-0.023645,-0.322256,-0.045531,-0.005922,0.033132
XLB,0.193065,0.016519,-0.019287,-0.007407,-0.000512,-0.013779,0.055900,-0.033101,-0.018465,-0.011226,-0.012923,-0.004275,0.074159,0.013884,-0.041826,0.007407,-0.149148,0.052878,-0.041179,0.029570
XLE,0.249276,0.024113,0.061032,-0.000006,0.014584,-0.005544,0.068929,0.010634,-0.007511,-0.008617,0.012285,-0.008752,0.108314,0.016194,-0.082898,0.032326,-0.127525,-0.040888,-0.037159,0.092151
XLF,0.316068,0.012246,-0.001223,-0.004557,-0.008886,-0.017443,0.122747,-0.045382,0.028220,-0.029369,-0.001016,0.002663,0.146797,0.035919,-0.306136,0.025029,0.078607,-0.001772,-0.032277,0.029337
XLI,0.211946,0.008900,-0.021664,-0.011165,-0.006091,-0.018660,0.060845,-0.031214,0.000816,-0.024639,-0.005345,-0.006689,0.058262,0.018356,-0.095225,0.002165,-0.074588,0.048944,-0.028933,0.040387
XLK,0.293204,0.020723,-0.028275,-0.001833,-0.009184,-0.025467,0.094112,-0.041215,-0.011223,-0.016288,-0.018925,-0.045519,0.071978,0.001561,0.026084,-0.015338,-0.257386,0.115534,-0.049189,-0.006225
XLP,0.195829,-0.004338,-0.010850,-0.010092,-0.006753,-0.015570,0.055368,-0.028291,0.013703,-0.012315,0.005556,-0.010184,0.065869,0.034208,-0.108598,-0.036139,0.043291,-0.059572,-0.000820,0.048493
XLU,0.264932,0.003075,-0.028135,-0.015532,0.003062,-0.011819,-0.045530,-0.020122,0.019405,-0.000697,0.020170,0.009889,-0.029610,0.038936,0.063400,-0.006961,0.064558,-0.084258,0.007584,0.018285
XLV,0.193789,0.001992,-0.003671,-0.009623,-0.012513,-0.010508,0.092950,-0.012427,-0.002285,-0.013144,-0.014776,-0.016615,0.055009,0.000959,-0.004015,-0.028727,-0.077220,0.006163,-0.008049,-0.018435


### 3.5 & 3.6 Comparison of Results

The following dataframe shows the R-squared values obtained from the 4 models fitted on each ETF.

In [23]:
r_sq_comp = pd.DataFrame()

for i in ['Xlei', 'Xlag', 'Xcei', 'Xall']:
    df = globals()[i + '_param_matrix']
    r_sq_comp['R-squared ' + i[1:].upper()] = df['R-squared']

r_sq_comp

,R-squared LEI,R-squared LAG,R-squared CEI,R-squared ALL
IYR,0.114582,0.040511,0.021508,0.188660
IYZ,0.046510,0.020704,0.004206,0.137117
XLB,0.103479,0.028873,0.034730,0.193065
XLE,0.104510,0.031102,0.083671,0.249276
XLF,0.170083,0.077047,0.041653,0.316068
XLI,0.139086,0.015905,0.037666,0.211946
XLK,0.143782,0.031851,0.019776,0.293204
XLP,0.119721,0.017474,0.003617,0.195829
XLU,0.148679,0.045125,0.029058,0.264932
XLV,0.149585,0.031734,0.010836,0.193789


The above table shows that the R-squared obtained from the regression of all indicators on the various responses is the highest, followed by the regression of the Leading Economic Indicators (LEI) on the responses.

However, it is important to note that the number of predictors is different for the 4 models, and that R-squared typically increases with an increase in the number of predictors, even if many of them are not statistically significantly related to the response. That said, the adjusted R-squared (formula below), which penalises models with large number of predictors, of the 4 models is compared below for each response.

$$
R_{a d j}^{2}=1-\left[\frac{\left(1-R^{2}\right)(n-1)}{n-k-1}\right]
$$

where n is the sample size and k is the number of independent regressors.

In [24]:
adj_r_sq_comp = pd.DataFrame()

param_matrix_names = ['Xlei', 'Xlag', 'Xcei', 'Xall']
regressors = [9, 6, 4, 19]

for i, j in zip(param_matrix_names, regressors):
    df = globals()[i + '_param_matrix']
    adj_r_sq_comp['Adj. R-squared ' + i[1:].upper()] = 1-((1 - df['R-squared']) * (72-1)/(72-j-1))

adj_r_sq_comp

,Adj. R-squared LEI,Adj. R-squared LAG,Adj. R-squared CEI,Adj. R-squared ALL
IYR,-0.013947,-0.048057,-0.036909,-0.107790
IYZ,-0.091900,-0.069693,-0.055244,-0.178168
XLB,-0.026661,-0.060770,-0.022898,-0.101777
XLE,-0.025480,-0.058335,0.028965,-0.025027
XLF,0.049611,-0.008149,-0.015561,0.066170
XLI,0.014115,-0.074935,-0.019787,-0.075997
XLK,0.019492,-0.057517,-0.038745,0.034952
XLP,-0.008061,-0.073221,-0.055869,-0.098003
XLU,0.025100,-0.043017,-0.028909,-0.003651
XLV,0.026137,-0.057644,-0.048218,-0.100788


For most of the ETFs, the adjusted R-squared across all 4 model types is negative.

## Part-4: Model Selection
### 4.1 Categorisation of ETF into LEI, LAG or CEI

For each ETF, the R-squared obtained from the regression of LEI, LAG or CEI on its returns is compared and the ETF is assigned to the category with the highest R-squared value. We do not use adjusted R-squared values here, since the 3 models have different predictors and adjusted R-squared is typically used to compare the performance of a model with an extended version of it which includes the predictors of the original model, in addition to new predictors. However, the context here is different from the one described previously, which is why we categorise ETFs based on R-squared and not adjusted R-squared.

In [25]:
categories = pd.DataFrame(index = r_sq_comp.index)
col_with_max = r_sq_comp[['R-squared LEI', 'R-squared LAG', 'R-squared CEI']].idxmax(axis=1)
categories['Category'] = [i.split()[1] for i in col_with_max]
categories

,Category
IYR,LEI
IYZ,LEI
XLB,LEI
XLE,LEI
XLF,LEI
XLI,LEI
XLK,LEI
XLP,LEI
XLU,LEI
XLV,LEI


## Part-5: Category 2 Models: LASSO
### 5.1 LASSO Regression with All Economic Indicators

We run a lasso regression of all indicators on each ETF with 10 different penalty values. This would give us 10 models per ETF, and 110 models in total.

In [26]:
penalty_values = np.linspace(0.1, 1.0, 10)
print('Penalty Values:', penalty_values)

Penalty Values: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [27]:
# Fitting the LASSO regression model with different penalty values
lasso_param_matrix = np.empty((0,19))
lasso_r_sq = np.empty((0,1))

for value in penalty_values.tolist():
    regr_lasso = Lasso(alpha = value, tol = 0.001)
    
    for k, v in logrtn_dict.items():
        Y = v
        regr_lasso.fit(Xall, Y)
    
        # Appending the relevant model parameters into matrices
        lasso_param_matrix = np.append(lasso_param_matrix, regr_lasso.coef_.reshape(1,19), axis = 0)
        lasso_r_sq = np.append(lasso_r_sq, np.array(regr_lasso.score(Xall, Y)).reshape(1,1), axis = 0)

In [28]:
lst1 = list(itertools.chain.from_iterable(itertools.repeat(x, 10) for x in etfs_lst))

lst2 = []
for i in range(11):
    for j in penalty_values:
        lst2.append(j)

In [29]:
# Summary table of relevant model parameters
col = Xall.columns
lasso_param_matrix = pd.DataFrame(lasso_param_matrix, columns=col)
lasso_param_matrix.insert(0, 'ETF', lst1)
lasso_param_matrix.insert(1, 'Penalty', lst2)
lasso_param_matrix.insert(2, 'R-squared', lasso_r_sq)

lasso_param_matrix.head(10)

,ETF,Penalty,R-squared,AWHAEMAN,IC4WSA,DGORDER,NEWORDER,PERMIT,SP500,NFCINONFINLEVERAGE,...,MNFCTRIRSA,UEMPMEAN,BUSLOANS,DPRIME,CUSR0000SASLE,CREDIT_BY_INC,PAYEMS,W875RX1,INDPRO,CMRMTSPL
0,IYR,0.1,0.077999,-0.0,1.057434e-07,-2.294168e-08,2.555033e-07,-0.000144,-0.000000,-0.0,...,0.0,-0.0,0.000000,0.0,0.0,0.0,0.000007,-0.000015,0.0,-5.822427e-08
1,IYR,0.2,0.057619,0.0,1.718511e-07,2.659556e-07,-1.719562e-06,-0.000070,0.000018,-0.0,...,0.0,-0.0,0.000133,0.0,0.0,0.0,-0.000008,-0.000009,0.0,3.127209e-07
2,IYR,0.3,0.077612,-0.0,1.148160e-06,-3.373012e-07,-2.035853e-06,-0.000056,0.000037,-0.0,...,0.0,0.0,0.000050,0.0,0.0,0.0,0.000004,-0.000045,-0.0,7.081175e-07
3,IYR,0.4,0.165028,0.0,2.313741e-06,-1.503977e-07,-9.584961e-07,-0.000027,0.000000,0.0,...,0.0,-0.0,0.000141,-0.0,-0.0,0.0,0.000008,-0.000136,-0.0,2.196156e-06
4,IYR,0.5,0.135550,-0.0,4.340349e-07,3.961789e-08,-4.348578e-06,-0.000061,0.000143,-0.0,...,0.0,-0.0,0.000183,-0.0,-0.0,0.0,-0.000006,-0.000106,-0.0,4.375841e-07
5,IYR,0.6,0.068857,-0.0,6.527628e-07,-4.532448e-07,-1.733494e-06,-0.000072,0.000028,-0.0,...,0.0,-0.0,0.000020,-0.0,0.0,0.0,0.000001,-0.000026,-0.0,6.399258e-07
6,IYR,0.7,0.090837,-0.0,6.284079e-07,3.152034e-08,-4.781728e-06,-0.000111,0.000103,-0.0,...,0.0,-0.0,0.000000,-0.0,0.0,0.0,-0.000000,-0.000013,-0.0,1.465765e-07
7,IYR,0.8,0.082162,-0.0,1.160034e-07,-3.880075e-07,5.854265e-07,-0.000085,0.000035,-0.0,...,0.0,-0.0,0.000067,0.0,0.0,-0.0,-0.000005,-0.000000,-0.0,1.955847e-07
8,IYR,0.9,0.155773,0.0,5.397971e-08,-9.190642e-07,5.804681e-06,-0.000074,-0.000054,-0.0,...,0.0,0.0,-0.000000,0.0,0.0,0.0,0.000017,-0.000033,0.0,-3.554432e-07
9,IYR,1.0,0.134214,-0.0,1.658293e-07,-4.745566e-07,-1.707600e-06,-0.000049,0.000129,-0.0,...,-0.0,-0.0,0.000000,0.0,0.0,-0.0,-0.000001,-0.000024,-0.0,-3.105981e-07


#### LASSO Regression with Leading Economic Indicators (LEI)

In [30]:
# Fitting the LASSO regression model with different penalty values
lasso_param_matrix_lei = np.empty((0,9))
lasso_r_sq_lei = np.empty((0,1))

for value in penalty_values.tolist():
    regr_lasso = Lasso(alpha = value, tol = 0.001)
    
    for k, v in logrtn_dict.items():
        Y = v
        regr_lasso.fit(Xlei, Y)
    
        # Appending the relevant model parameters into matrices
        lasso_param_matrix_lei = np.append(lasso_param_matrix_lei, regr_lasso.coef_.reshape(1,9), axis = 0)
        lasso_r_sq_lei = np.append(lasso_r_sq_lei, np.array(regr_lasso.score(Xlei, Y)).reshape(1,1), axis = 0)

In [31]:
# Summary table of relevant model parameters
col = Xlei.columns
lasso_param_matrix_lei = pd.DataFrame(lasso_param_matrix_lei, columns=col)
lasso_param_matrix_lei.insert(0, 'ETF', lst1)
lasso_param_matrix_lei.insert(1, 'Penalty', lst2)
lasso_param_matrix_lei.insert(2, 'R-squared', lasso_r_sq_lei)

lasso_param_matrix_lei.head(10)

,ETF,Penalty,R-squared,AWHAEMAN,IC4WSA,DGORDER,NEWORDER,PERMIT,SP500,NFCINONFINLEVERAGE,DGS10_FEDFUNDS,UMCSENT
0,IYR,0.1,0.061447,-0.0,-2.698928e-07,1.017632e-08,-0.000000e+00,-0.000126,0.000008,-0.0,-0.0,-0.0
1,IYR,0.2,0.026151,-0.0,-2.731744e-08,3.176422e-07,-2.302493e-06,-0.000070,0.000012,-0.0,-0.0,-0.0
2,IYR,0.3,0.046124,-0.0,5.397561e-07,-2.480351e-07,-3.349303e-06,-0.000060,0.000071,-0.0,-0.0,-0.0
3,IYR,0.4,0.028982,0.0,7.630827e-07,7.997773e-08,-4.044470e-06,-0.000040,0.000078,0.0,-0.0,-0.0
4,IYR,0.5,0.073172,-0.0,8.228994e-07,1.435971e-07,-5.770236e-06,-0.000069,0.000098,-0.0,0.0,-0.0
5,IYR,0.6,0.049753,-0.0,2.260493e-07,-4.080233e-07,-2.240618e-06,-0.000076,0.000051,-0.0,-0.0,-0.0
6,IYR,0.7,0.088151,0.0,6.450912e-07,3.660574e-08,-4.792810e-06,-0.000113,0.000101,-0.0,-0.0,-0.0
7,IYR,0.8,0.062626,-0.0,8.871468e-09,-3.686369e-07,4.561870e-07,-0.000085,0.000031,-0.0,-0.0,-0.0
8,IYR,0.9,0.093907,-0.0,-3.877782e-07,-8.180868e-07,3.476043e-06,-0.000068,-0.000009,-0.0,-0.0,-0.0
9,IYR,1.0,0.105674,0.0,7.806381e-07,-4.957971e-07,-1.429342e-06,-0.000051,0.000091,-0.0,0.0,-0.0


#### LASSO Regression with Lagging Economic Indicators (LAG)

In [32]:
# Fitting the LASSO regression model with different penalty values
lasso_param_matrix_lag = np.empty((0,6))
lasso_r_sq_lag = np.empty((0,1))

for value in penalty_values.tolist():
    regr_lasso = Lasso(alpha = value, tol = 0.001)
    
    for k, v in logrtn_dict.items():
        Y = v
        regr_lasso.fit(Xlag, Y)
    
        # Appending the relevant model parameters into matrices
        lasso_param_matrix_lag = np.append(lasso_param_matrix_lag, regr_lasso.coef_.reshape(1,6), axis = 0)
        lasso_r_sq_lag = np.append(lasso_r_sq_lag, np.array(regr_lasso.score(Xlag, Y)).reshape(1,1), axis = 0)

In [33]:
# Summary table of relevant model parameters
col = Xlag.columns
lasso_param_matrix_lag = pd.DataFrame(lasso_param_matrix_lag, columns=col)
lasso_param_matrix_lag.insert(0, 'ETF', lst1)
lasso_param_matrix_lag.insert(1, 'Penalty', lst2)
lasso_param_matrix_lag.insert(2, 'R-squared', lasso_r_sq_lag)

lasso_param_matrix_lag.head(10)

,ETF,Penalty,R-squared,MNFCTRIRSA,UEMPMEAN,BUSLOANS,DPRIME,CUSR0000SASLE,CREDIT_BY_INC
0,IYR,0.1,0.001537,-0.0,0.0,-4.872667e-06,0.0,-0.0,-0.0
1,IYR,0.2,0.000158,0.0,0.0,9.840474e-07,-0.0,-0.0,-0.0
2,IYR,0.3,0.002044,0.0,0.0,7.456180e-06,-0.0,0.0,0.0
3,IYR,0.4,0.001112,0.0,0.0,6.878776e-06,-0.0,0.0,0.0
4,IYR,0.5,0.002555,0.0,0.0,8.395302e-06,-0.0,0.0,0.0
5,IYR,0.6,0.002381,0.0,0.0,7.526762e-06,-0.0,0.0,0.0
6,IYR,0.7,0.006636,0.0,-0.0,1.365686e-05,-0.0,0.0,0.0
7,IYR,0.8,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,-0.0
8,IYR,0.9,0.000000,0.0,0.0,-0.000000e+00,0.0,-0.0,-0.0
9,IYR,1.0,0.001510,-0.0,0.0,-4.925065e-06,0.0,0.0,0.0


#### LASSO Regression with Coincidewith Coincidental Economic Indicators (CEI)ntal Economic Indicators (CEI)

In [34]:
# Fitting the LASSO regression model with different penalty values
lasso_param_matrix_cei = np.empty((0,4))
lasso_r_sq_cei = np.empty((0,1))

for value in penalty_values.tolist():
    regr_lasso = Lasso(alpha = value, tol = 0.001)
    
    for k, v in logrtn_dict.items():
        Y = v
        regr_lasso.fit(Xcei, Y)
    
        # Appending the relevant model parameters into matrices
        lasso_param_matrix_cei = np.append(lasso_param_matrix_cei, regr_lasso.coef_.reshape(1,4), axis = 0)
        lasso_r_sq_cei = np.append(lasso_r_sq_cei, np.array(regr_lasso.score(Xcei, Y)).reshape(1,1), axis = 0)

In [35]:
# Summary table of relevant model parameters
col = Xcei.columns
lasso_param_matrix_cei = pd.DataFrame(lasso_param_matrix_cei, columns=col)
lasso_param_matrix_cei.insert(0, 'ETF', lst1)
lasso_param_matrix_cei.insert(1, 'Penalty', lst2)
lasso_param_matrix_cei.insert(2, 'R-squared', lasso_r_sq_cei)

lasso_param_matrix_cei.head(10)

,ETF,Penalty,R-squared,PAYEMS,W875RX1,INDPRO,CMRMTSPL
0,IYR,0.1,0.003923,1.140356e-06,-0.000008,0.0,-3.063027e-08
1,IYR,0.2,0.002579,6.728704e-07,-0.000012,-0.0,9.568377e-08
2,IYR,0.3,0.020335,5.939669e-06,-0.000064,-0.0,3.605356e-07
3,IYR,0.4,0.072029,7.339020e-06,-0.000153,-0.0,1.441582e-06
4,IYR,0.5,0.036283,1.137530e-05,-0.000092,-0.0,2.500971e-07
5,IYR,0.6,0.017622,3.003098e-06,-0.000048,-0.0,4.158841e-07
6,IYR,0.7,0.012381,3.140073e-06,-0.000024,-0.0,1.174132e-07
7,IYR,0.8,0.002960,-2.021307e-06,0.000000,0.0,1.841864e-07
8,IYR,0.9,0.010624,4.523838e-06,-0.000004,0.0,-3.660354e-07
9,IYR,1.0,0.001430,1.044882e-06,-0.000000,0.0,-1.144374e-07


### 5.2 Results Comparison
#### Comparison of R-Squared for Linear Model with All Indicators vs Lasso Model with All Indicators

In [36]:
lin_lasso_comp = pd.DataFrame()
lin_lasso_comp['R-Squared Linear'] = r_sq_comp['R-squared ALL']

lst3 = []
for i in lasso_param_matrix.loc[lasso_param_matrix['Penalty'] == penalty_values[4]]['R-squared']:
    lst3.append(i)
lin_lasso_comp['R-Squared Lasso'] = lst3

lin_lasso_comp

,R-Squared Linear,R-Squared Lasso
IYR,0.188660,0.135550
IYZ,0.137117,0.149100
XLB,0.193065,0.049317
XLE,0.249276,0.008757
XLF,0.316068,0.013005
XLI,0.211946,0.023132
XLK,0.293204,0.084415
XLP,0.195829,0.081718
XLU,0.264932,0.024268
XLV,0.193789,0.020461


The above analysis shows that by introducing a term which penalises the coefficients of predictors, the coefficients of some predictors reduce to zero and the total variation in returns explained by the lasso model (R-squared) also declines relative to that explained by the linear model.

The rank ordering of the ETFs' R-squared (shown below) produced by the linear model is different from the one produced by the lasso model. So, the two models lead to different conclusions.

In [37]:
pd.DataFrame(lin_lasso_comp['R-Squared Linear']).sort_values('R-Squared Linear', ascending = False).index

Index(['XLF', 'XLK', 'XLU', 'XLE', 'XLI', 'XLY', 'XLP', 'XLV', 'XLB', 'IYR',
       'IYZ'],
      dtype='object')

In [38]:
pd.DataFrame(lin_lasso_comp['R-Squared Lasso']).sort_values('R-Squared Lasso', ascending = False).index

Index(['IYZ', 'IYR', 'XLK', 'XLP', 'XLB', 'XLY', 'XLU', 'XLI', 'XLV', 'XLF',
       'XLE'],
      dtype='object')

#### R-Squared for Lasso Model with LEI, LAG and CEI

In [39]:
r_sq_comp_lasso = pd.DataFrame()

for i in ['lei', 'lag', 'cei']:
    df = globals()['lasso_param_matrix_' + i]
    r_sq_comp_lasso['R-squared ' + i.upper()] = df.loc[df['Penalty'] == penalty_values[4]]['R-squared']
        
r_sq_comp_lasso['ETF'] = [i.split()[0].upper() for i in logrtn_dict]
r_sq_comp_lasso.set_index('ETF', drop = True, inplace = True)
r_sq_comp_lasso

,R-squared LEI,R-squared LAG,R-squared CEI
ETF,,,
IYR,0.073172,0.002555,0.036283
IYZ,0.023793,0.000914,0.069930
XLB,0.030541,0.001194,0.011287
XLE,0.006931,0.000000,0.000100
XLF,0.008274,0.000000,0.002266
XLI,0.011122,0.000000,0.015968
XLK,0.073675,0.000000,0.000947
XLP,0.064817,0.000000,0.008954
XLU,0.021283,0.000000,0.001749


In [40]:
categories_lasso = pd.DataFrame(index = r_sq_comp_lasso.index)
col_with_max = r_sq_comp_lasso[['R-squared LEI', 'R-squared LAG', 'R-squared CEI']].idxmax(axis=1)
categories_lasso['Category'] = [i.split()[1] for i in col_with_max]
categories_lasso

,Category
ETF,
IYR,LEI
IYZ,CEI
XLB,LEI
XLE,LEI
XLF,LEI
XLI,CEI
XLK,LEI
XLP,LEI
XLU,LEI


Linear and Lasso regressions place all ETFs into the LEI category except IYZ and XLI which the linear model places in the LEI category and the lasso model places in the CEI category.

## Part-6: Category 3 Models: Clustering Analysis
### 6.1 Distance Matrix and K-Means with K = 3

The code below computes the pairwise distances among the various datapoints for each ETF, which is followed by a computation of the distance correlation which, unlike Pearson's correlation, captures both linear and non-linear association between two variables.

In [41]:
#Distance matrix
distances_dict = {}

for k, v in logrtn_dict.items():
    distances_dict[str(k) + ' distance matrix'] = pairwise_distances(v)

#distances_dict['iyr log return distance matrix'].shape

In [42]:
#Distance correlation
dist_corr = pd.DataFrame(index = [k for k, v in logrtn_dict.items()], columns = [k for k, v in logrtn_dict.items()])
for k1, v1 in logrtn_dict.items():
    for k2, v2 in logrtn_dict.items():
        dist_corr.loc[k1][k2] = dcor.distance_correlation(v1, v2)

dist_corr

,iyr log return,iyz log return,xlb log return,xle log return,xlf log return,xli log return,xlk log return,xlp log return,xlu log return,xlv log return,xly log return
iyr log return,1.0,0.364468,0.327421,0.27978,0.299049,0.366434,0.375585,0.544865,0.629949,0.501191,0.434098
iyz log return,0.364468,1.0,0.614839,0.492231,0.472404,0.546916,0.548503,0.412458,0.260671,0.499889,0.57113
xlb log return,0.327421,0.614839,1.0,0.685077,0.697103,0.823083,0.675631,0.405641,0.162361,0.596323,0.73264
xle log return,0.27978,0.492231,0.685077,1.0,0.549166,0.595779,0.452804,0.301993,0.153785,0.359273,0.537837
xlf log return,0.299049,0.472404,0.697103,0.549166,1.0,0.774958,0.611338,0.431736,0.170647,0.605111,0.672079
xli log return,0.366434,0.546916,0.823083,0.595779,0.774958,1.0,0.681413,0.539493,0.179869,0.655141,0.746988
xlk log return,0.375585,0.548503,0.675631,0.452804,0.611338,0.681413,1.0,0.493577,0.242756,0.539397,0.748096
xlp log return,0.544865,0.412458,0.405641,0.301993,0.431736,0.539493,0.493577,1.0,0.533488,0.478115,0.577819
xlu log return,0.629949,0.260671,0.162361,0.153785,0.170647,0.179869,0.242756,0.533488,1.0,0.212626,0.212824
xlv log return,0.501191,0.499889,0.596323,0.359273,0.605111,0.655141,0.539397,0.478115,0.212626,1.0,0.657767


In [43]:
#K-Means
lst_for_Yall = []
for k, v in logrtn_dict.items():
    v = v.rename(columns={'Adj Close': k})
    lst_for_Yall.append(v)
    
Yall = reduce(lambda left, right: pd.merge(left, right, on=['Date'], how='outer'), lst_for_Yall)
Yall

,iyr log return,iyz log return,xlb log return,xle log return,xlf log return,xli log return,xlk log return,xlp log return,xlu log return,xlv log return,xly log return
Date,,,,,,,,,,,
2014-01-01,0.043145,-0.022691,-0.042038,-0.055060,-0.032592,-0.038528,-0.021161,-0.045461,0.039974,0.013445,-0.057880
2014-02-01,0.046416,-0.001736,0.066750,0.049944,0.029937,0.039774,0.043002,0.038267,0.033944,0.060330,0.062028
2014-03-01,-0.009999,0.040850,0.004239,0.015959,0.029066,0.005173,0.000000,0.016626,0.024662,-0.016112,-0.032231
2014-04-01,0.041294,-0.016709,0.012201,0.056121,-0.013457,0.017635,0.007041,0.032101,0.049641,-0.002892,-0.010549
2014-05-01,0.027585,0.027574,0.029150,0.016610,0.014916,0.019052,0.036896,0.017922,-0.010937,0.028328,0.028414
...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,0.033767,-0.054179,-0.028690,-0.086925,-0.048224,-0.026828,-0.015471,0.021474,0.049629,-0.005942,-0.009488
2019-09-01,0.011613,0.028484,0.026287,0.029485,0.039335,0.024383,0.012496,0.011462,0.033934,-0.005201,0.009323
2019-10-01,0.014779,0.025236,0.004843,-0.012064,0.029778,0.016558,0.041361,0.001657,0.000106,0.054134,0.004554


In [44]:
kmeans_data = Yall.to_numpy().reshape(11, 72)

In [45]:
kmeans = KMeans(n_clusters=3, random_state=1).fit(kmeans_data)
kmeans.cluster_centers_

array([[ 2.44659808e-02, -8.73448804e-02, -8.57771700e-02,
        -7.78477677e-02, -1.42147265e-01, -1.24199563e-01,
        -1.18869254e-01, -9.18135792e-02, -1.02481236e-01,
        -4.92219457e-02, -1.02733105e-01, -8.68177615e-02,
         1.14530790e-01,  6.21637832e-02,  6.09883076e-02,
         1.15577477e-01,  9.14765333e-02,  1.14356958e-01,
         7.14073083e-02,  5.92302615e-02,  4.26363475e-02,
         5.14951633e-02,  9.80181304e-02,  7.27538073e-03,
         5.71782208e-02,  3.13679867e-02,  2.27863774e-02,
         2.21128878e-02,  6.17304715e-02,  6.68181801e-02,
         1.76371219e-02,  4.02667829e-02,  1.07503808e-02,
         1.30603261e-02,  3.37668965e-02,  1.68266421e-03,
         8.14110115e-03,  1.32453032e-02, -3.10192138e-02,
        -1.73091990e-02,  4.33588218e-02,  3.20534551e-02,
         2.01430004e-02,  1.09045551e-03,  3.24940065e-02,
         6.63631518e-03,  2.52142473e-02,  3.88422204e-02,
         7.66112611e-03,  9.11306356e-02,  4.46965415e-0

In [46]:
print('Categories Assigned to Each ETF:', kmeans.labels_)
print('Inertia:', kmeans.inertia_)
print('No. of iterations:', kmeans.n_iter_)

Categories Assigned to Each ETF: [1 1 1 2 1 1 1 1 1 0 1]
Inertia: 0.7863740652925243
No. of iterations: 2


### 6.2 Results Comparison

Distance correlation is a robust and universal way to check if there is a relation, linear or non-linear, between two numeric variables. For example, if we have a set of pairs of numbers: (x1, y1) (x2, y2) ... (xn, yn), we can use distance correlation to check if there is any (not necessarily linear) relation between the two variables (x and y). Moreover, x and y can be vectors of different dimensions.

To calculate distance correlation, we first use 𝑥𝑖 to calculate distance matrix. Then we calculate distance matrix using 𝑦𝑖. The two distance matrices will have the same dimensions because the number of 𝑥𝑖 and 𝑦𝑖 is the same (because they come in pairs). This gives us a set of pairs of distances and we can use it to calculate distance correlation (correlation between distances).

The above matrix shows that there is a strong relationship between a majority of the ETFs. Because of this, one can imagine that returns across the 11 ETFs are governed by a common set of underlying factors.

However, the linear model shows that the 19 economic indicators combined explain only a limited part of the total variability (13-30%) in the ETFs' returns.

The above findings could imply the following:
- either the selected economic indicators are not a good choice of predictors to explain the ETFs' returns or
- if they are, the true underlying relationship between the ETFs' returns and the economic indicators is non-linear

## Part-7: Category 4 Models: Regression Trees
### 7.1 Regression Tree with All Economic Indicators

In [47]:
# Running the Regression Tree model with various depths
depths_scores_reg = {}

for k, v in logrtn_dict.items():
    Y = v
    depths_scores_reg[str(k)] = np.zeros((5, 2))
    
    for d in range(1, 6):
        tree_reg = DecisionTreeRegressor(criterion = 'squared_error', max_depth = d, random_state = 1)
        tree_reg.fit(Xall, Y)

        depths_scores_reg[str(k)][d-1, 0] = tree_reg.get_depth()
        depths_scores_reg[str(k)][d-1, 1] = tree_reg.score(Xall, Y)
        
depths_scores_reg

{'iyr log return': array([[1.        , 0.07400676],
        [2.        , 0.25734299],
        [3.        , 0.40606138],
        [4.        , 0.52038085],
        [5.        , 0.64717331]]),
 'iyz log return': array([[1.        , 0.07498072],
        [2.        , 0.25850292],
        [3.        , 0.34525013],
        [4.        , 0.5094311 ],
        [5.        , 0.69654616]]),
 'xlb log return': array([[1.        , 0.07313294],
        [2.        , 0.23834315],
        [3.        , 0.54901449],
        [4.        , 0.68744205],
        [5.        , 0.78726648]]),
 'xle log return': array([[1.        , 0.10233497],
        [2.        , 0.20975104],
        [3.        , 0.42663226],
        [4.        , 0.5839626 ],
        [5.        , 0.67015302]]),
 'xlf log return': array([[1.        , 0.11563081],
        [2.        , 0.20938693],
        [3.        , 0.42005719],
        [4.        , 0.58125497],
        [5.        , 0.67817624]]),
 'xli log return': array([[1.        , 0.13545323]

In [48]:
# Summary of the performance-maximising/ optimal depths for various ETFs
depth_summary_reg = pd.DataFrame(columns = ['ETF', 'Optimal Depth(s)', 'Maximum Prediction Score'])

for k, v in logrtn_dict.items():
    idx_reg = np.where(depths_scores_reg[str(k)][:, 1] == np.amax(depths_scores_reg[str(k)][:, 1]))[0]
    depth_summary_reg.loc[len(depth_summary_reg)] = [str(k).split()[0].upper(), depths_scores_reg[str(k)][idx_reg, 0][0], max(depths_scores_reg[str(k)][:, 1])]
    
depth_summary_reg

,ETF,Optimal Depth(s),Maximum Prediction Score
0,IYR,5.0,0.647173
1,IYZ,5.0,0.696546
2,XLB,5.0,0.787266
3,XLE,5.0,0.670153
4,XLF,5.0,0.678176
5,XLI,5.0,0.712876
6,XLK,5.0,0.585768
7,XLP,5.0,0.827359
8,XLU,5.0,0.500362
9,XLV,5.0,0.504710


#### Regression Tree with Leading Economic Indicators (LEI)

In [49]:
# Running the Regression Tree model with various depths
depths_scores_reg_lei = {}

for k, v in logrtn_dict.items():
    Y = v
    depths_scores_reg_lei[str(k)] = np.zeros((5, 2))
    
    for d in range(1, 6):
        tree_reg = DecisionTreeRegressor(criterion = 'squared_error', max_depth = d, random_state = 1)
        tree_reg.fit(Xlei, Y)

        depths_scores_reg_lei[str(k)][d-1, 0] = tree_reg.get_depth()
        depths_scores_reg_lei[str(k)][d-1, 1] = tree_reg.score(Xlei, Y)
        
depths_scores_reg_lei

{'iyr log return': array([[1.        , 0.07327706],
        [2.        , 0.26450518],
        [3.        , 0.48367336],
        [4.        , 0.62373207],
        [5.        , 0.71789183]]),
 'iyz log return': array([[1.        , 0.07498072],
        [2.        , 0.22772514],
        [3.        , 0.40729004],
        [4.        , 0.63038796],
        [5.        , 0.74976986]]),
 'xlb log return': array([[1.        , 0.07313294],
        [2.        , 0.23834315],
        [3.        , 0.52154123],
        [4.        , 0.65216915],
        [5.        , 0.74054283]]),
 'xle log return': array([[1.        , 0.06935396],
        [2.        , 0.19500808],
        [3.        , 0.41658041],
        [4.        , 0.57904927],
        [5.        , 0.66815085]]),
 'xlf log return': array([[1.        , 0.10773216],
        [2.        , 0.19060602],
        [3.        , 0.38364001],
        [4.        , 0.61432173],
        [5.        , 0.78640962]]),
 'xli log return': array([[1.        , 0.13545323]

In [50]:
# Summary of the performance-maximising/ optimal depths for various ETFs
depth_summary_reg_lei = pd.DataFrame(columns = ['ETF', 'Optimal Depth(s)', 'Maximum Prediction Score'])

for k, v in logrtn_dict.items():
    idx_reg_lei = np.where(depths_scores_reg_lei[str(k)][:, 1] == np.amax(depths_scores_reg_lei[str(k)][:, 1]))[0]
    depth_summary_reg_lei.loc[len(depth_summary_reg_lei)] = [str(k).split()[0].upper(), depths_scores_reg_lei[str(k)][idx_reg_lei, 0][0], max(depths_scores_reg_lei[str(k)][:, 1])]
    
depth_summary_reg_lei

,ETF,Optimal Depth(s),Maximum Prediction Score
0,IYR,5.0,0.717892
1,IYZ,5.0,0.749770
2,XLB,5.0,0.740543
3,XLE,5.0,0.668151
4,XLF,5.0,0.786410
5,XLI,5.0,0.800414
6,XLK,5.0,0.580392
7,XLP,5.0,0.802224
8,XLU,5.0,0.501709
9,XLV,5.0,0.504710


#### Regression Tree with Lagging Economic Indicators (LAG)

In [51]:
# Running the Regression Tree model with various depths
depths_scores_reg_lag = {}

for k, v in logrtn_dict.items():
    Y = v
    depths_scores_reg_lag[str(k)] = np.zeros((5, 2))
    
    for d in range(1, 6):
        tree_reg = DecisionTreeRegressor(criterion = 'squared_error', max_depth = d, random_state = 1)
        tree_reg.fit(Xlag, Y)

        depths_scores_reg_lag[str(k)][d-1, 0] = tree_reg.get_depth()
        depths_scores_reg_lag[str(k)][d-1, 1] = tree_reg.score(Xlag, Y)
        
depths_scores_reg_lag

{'iyr log return': array([[1.        , 0.07400676],
        [2.        , 0.25734299],
        [3.        , 0.39909727],
        [4.        , 0.49929477],
        [5.        , 0.58106271]]),
 'iyz log return': array([[1.        , 0.05806836],
        [2.        , 0.1987225 ],
        [3.        , 0.30918557],
        [4.        , 0.38263184],
        [5.        , 0.47620692]]),
 'xlb log return': array([[1.        , 0.04845261],
        [2.        , 0.22890366],
        [3.        , 0.40390761],
        [4.        , 0.49195812],
        [5.        , 0.63727946]]),
 'xle log return': array([[1.        , 0.05117997],
        [2.        , 0.15324384],
        [3.        , 0.29120318],
        [4.        , 0.47530147],
        [5.        , 0.60269335]]),
 'xlf log return': array([[1.        , 0.04967824],
        [2.        , 0.09954496],
        [3.        , 0.27017068],
        [4.        , 0.46254169],
        [5.        , 0.60240448]]),
 'xli log return': array([[1.        , 0.05752922]

In [52]:
# Summary of the performance-maximising/ optimal depths for various ETFs
depth_summary_reg_lag = pd.DataFrame(columns = ['ETF', 'Optimal Depth(s)', 'Maximum Prediction Score'])

for k, v in logrtn_dict.items():
    idx_reg_lag = np.where(depths_scores_reg_lag[str(k)][:, 1] == np.amax(depths_scores_reg_lag[str(k)][:, 1]))[0]
    depth_summary_reg_lag.loc[len(depth_summary_reg_lag)] = [str(k).split()[0].upper(), depths_scores_reg_lag[str(k)][idx_reg_lag, 0][0], max(depths_scores_reg_lag[str(k)][:, 1])]
    
depth_summary_reg_lag

,ETF,Optimal Depth(s),Maximum Prediction Score
0,IYR,5.0,0.581063
1,IYZ,5.0,0.476207
2,XLB,5.0,0.637279
3,XLE,5.0,0.602693
4,XLF,5.0,0.602404
5,XLI,5.0,0.709075
6,XLK,5.0,0.667930
7,XLP,5.0,0.430558
8,XLU,5.0,0.542921
9,XLV,5.0,0.418976


#### Regression Tree with Coincidental Economic Indicators (CEI)

In [53]:
# Running the Regression Tree model with various depths
depths_scores_reg_cei = {}

for k, v in logrtn_dict.items():
    Y = v
    depths_scores_reg_cei[str(k)] = np.zeros((5, 2))
    
    for d in range(1, 6):
        tree_reg = DecisionTreeRegressor(criterion = 'squared_error', max_depth = d, random_state = 1)
        tree_reg.fit(Xcei, Y)

        depths_scores_reg_cei[str(k)][d-1, 0] = tree_reg.get_depth()
        depths_scores_reg_cei[str(k)][d-1, 1] = tree_reg.score(Xcei, Y)
        
depths_scores_reg_cei

{'iyr log return': array([[1.        , 0.06188227],
        [2.        , 0.10230869],
        [3.        , 0.18307405],
        [4.        , 0.34227287],
        [5.        , 0.57389097]]),
 'iyz log return': array([[1.        , 0.04487586],
        [2.        , 0.08695479],
        [3.        , 0.15709427],
        [4.        , 0.22611758],
        [5.        , 0.31014772]]),
 'xlb log return': array([[1.        , 0.05505304],
        [2.        , 0.10449588],
        [3.        , 0.17528235],
        [4.        , 0.39400797],
        [5.        , 0.5485079 ]]),
 'xle log return': array([[1.        , 0.10233497],
        [2.        , 0.2014947 ],
        [3.        , 0.27083873],
        [4.        , 0.34966568],
        [5.        , 0.40554661]]),
 'xlf log return': array([[1.        , 0.11563081],
        [2.        , 0.18597037],
        [3.        , 0.30710584],
        [4.        , 0.44594666],
        [5.        , 0.51160985]]),
 'xli log return': array([[1.        , 0.07668246]

In [54]:
# Summary of the performance-maximising/ optimal depths for various ETFs
depth_summary_reg_cei = pd.DataFrame(columns = ['ETF', 'Optimal Depth(s)', 'Maximum Prediction Score'])

for k, v in logrtn_dict.items():
    idx_reg_cei = np.where(depths_scores_reg_cei[str(k)][:, 1] == np.amax(depths_scores_reg_cei[str(k)][:, 1]))[0]
    depth_summary_reg_cei.loc[len(depth_summary_reg_cei)] = [str(k).split()[0].upper(), depths_scores_reg_cei[str(k)][idx_reg_cei, 0][0], max(depths_scores_reg_cei[str(k)][:, 1])]
    
depth_summary_reg_cei

,ETF,Optimal Depth(s),Maximum Prediction Score
0,IYR,5.0,0.573891
1,IYZ,5.0,0.310148
2,XLB,5.0,0.548508
3,XLE,5.0,0.405547
4,XLF,5.0,0.511610
5,XLI,5.0,0.587755
6,XLK,5.0,0.568089
7,XLP,5.0,0.383161
8,XLU,5.0,0.287469
9,XLV,5.0,0.327995


### 7.2 Results Comparison
#### Comparison of Linear and Lasso Models with All Indicators vs. Regression Tree with All Indicators

The regression tree does a much better job of fitting the datasets compared with linear and lasso regressions. The reason for this is that the former is a non-parametric method whereas the latter are parametric methods.

The parametric approach is a model-based approach; it reduces the problem of estimating the underlying function down to one of estimating a set of parameters. Assuming a parametric form for the function simplifies the problem of estimating it because it is generally much easier to estimate a set of parameters than it is to fit an entirely arbitrary function. The potential disadvantage of a parametric approach is that the model we choose will usually not match the true unknown form of f. This is what is happening in case of linear and lasso regressions. 

Because the chosen model is too far from the true f, our estimate is not great. We can try to address this problem by choosing flexible models that can fit many different possible functional forms for f. These come under the purview of non-parametric approaches, one of which is the regression tree. But in general, fitting a more flexible and complex model can lead to a phenomenon known as overfitting the data, which essentially means they follow the errors, or noise, too closely.

#### Prediction Score for Regression Trees with LEI, LAG and CEI

In [55]:
pred_score_comp_tree = pd.DataFrame()

for i in ['lei', 'lag', 'cei']:
    df = globals()['depth_summary_reg_' + i]
    pred_score_comp_tree['Max. Score ' + i.upper()] = df['Maximum Prediction Score']
        
pred_score_comp_tree['ETF'] = [i.split()[0].upper() for i in logrtn_dict]
pred_score_comp_tree.set_index('ETF', drop = True, inplace = True)
pred_score_comp_tree

,Max. Score LEI,Max. Score LAG,Max. Score CEI
ETF,,,
IYR,0.717892,0.581063,0.573891
IYZ,0.749770,0.476207,0.310148
XLB,0.740543,0.637279,0.548508
XLE,0.668151,0.602693,0.405547
XLF,0.786410,0.602404,0.511610
XLI,0.800414,0.709075,0.587755
XLK,0.580392,0.667930,0.568089
XLP,0.802224,0.430558,0.383161
XLU,0.501709,0.542921,0.287469


In [56]:
categories_tree = pd.DataFrame(index = pred_score_comp_tree.index)
col_with_max = pred_score_comp_tree[['Max. Score LEI', 'Max. Score LAG', 'Max. Score CEI']].idxmax(axis=1)
categories_tree['Category'] = [i.split()[-1] for i in col_with_max]
categories_tree

,Category
ETF,
IYR,LEI
IYZ,LEI
XLB,LEI
XLE,LEI
XLF,LEI
XLI,LEI
XLK,LAG
XLP,LEI
XLU,LAG


Regression tree places all ETFs into LEI category except XLK and XLU which it places in the LAG category

## Part-8: Interpreting the Models

- Linear regression places all ETFs into the LEI category 
- Lasso regression places all ETFs into LEI category except IYZ and XLI which it places in the CEI category
- Regression tree places all ETFs into LEI category except XLK and XLU which it places in the LAG category

In [57]:
categories_summary = pd.DataFrame(index = categories_tree.index)
categories_summary['Linear Model'] = categories['Category']
categories_summary['Lasso Model'] = categories_lasso['Category']
categories_summary['Regression Tree'] = categories_tree['Category']
categories_summary

,Linear Model,Lasso Model,Regression Tree
ETF,,,
IYR,LEI,LEI,LEI
IYZ,LEI,CEI,LEI
XLB,LEI,LEI,LEI
XLE,LEI,LEI,LEI
XLF,LEI,LEI,LEI
XLI,LEI,CEI,LEI
XLK,LEI,LEI,LAG
XLP,LEI,LEI,LEI
XLU,LEI,LEI,LAG


### 8.1 Sectors in the LEI Bucket

All four models (linear regression, lasso regression, regression tree and K-means clustering) place a majority of the ETFs into the LEI bucket. This is natural because stock prices are inherently forward-looking and, therefore, reflect and are explained well by the trends in leading economic indicators. 

### 8.2 Sectors in the CEI Bucket

While linear regression and regression tree categorise IYZ (Telecommunications) and XLI (Industrials) into the LEI bucket, the lasso model puts them in the CEI bucket. The common thing between both telecommunications and industrials is that they require heavy capital investment (network towers and connections in case of Telecommunications and heavy machinery in case of Industrials). Since manufacturing and communications often form the core of many economies like the US with classic historical evolutions (dominance of agriculture followed by dominance of manufacturing and services in % contribution to GDP), the economic state of these sectors can significantly influence the the state of the overall economy. Stated alternatively, the overall economy goes hand-in-hand with these sectors. For this reason, coincident economic indicators did a better job of predicting their returns under the lasso model.

### 8.3 Sectors in the LAG Bucket

While linear and lasso regressions categorise XLK (Technology), XLU (Utilities) and XLY (Consumer Discretionary) into the LEI bucket, the regression tree puts them in the LAG bucket. Although it is a bit odd for returns in the Tech sector to be explained better by LAG than by LEI or CEI, Utilities and Consumer Discretionary sectors are defensive and less sensitive to boom-bust cycles in the macroeconomy. This is why LEI and CEI, which better capture these boom-bust cycles than LAG, do a poorer job than LAG to explain returns in these sectors.